<a href="https://colab.research.google.com/github/ajasja/RosettaCrashCourse/blob/main/notebooks/W03_layer_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install pyrosetta 
#@markdown Run cells by clicking on the little ▶ play icon to the left of each cell or by going to Runtime->Run all. 

#@markdown Enter the user name and password obtained from https://els2.comotion.uw.edu/product/pyrosetta

#@markdown The notebook must be connected to Google Drive. 

#@markdown Author: Ajasja Ljubetič (ajasja.ljubetic@gmail.com), inspired by https://nbviewer.org/github/RosettaCommons/PyRosetta.notebooks


#only needed for colabfold
!pip install pyrosettacolabsetup py3dmol https://github.com/RosettaCommons/pyrosetta_viewer3d/archive/main.zip --quiet
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()

In [ ]:
#@title Helper functions
def view_single(pose):
  import py3Dmol
  import pyrosetta.distributed.io as io
  view = py3Dmol.view(width=500, height=500)
  view.addModel(io.to_pdbstring(pose),'pdb')
  view.setStyle({'model': 0}, {"cartoon": True})
  view.addStyle({'model': 0}, {"stick": True})
  view.zoomTo()
  view.show()
  return None

def compare_poses(pose1, pose2):
  import py3Dmol
  import pyrosetta.distributed.io as io
  view = py3Dmol.view(width=500, height=500)
  view.addModel(io.to_pdbstring(pose1),'pdb')
  view.addModel(io.to_pdbstring(pose2),'pdb')
  view.setStyle({'model': 0}, {"cartoon": {'color': 'orange'}})
  view.addStyle({'model': 0}, {"stick": {'color': 'orange'}})
  view.setStyle({'model': 1}, {"cartoon": {'color': 'blue'}})
  view.addStyle({'model': 1}, {"stick": {'color': 'blue'}})
  view.zoomTo()
  view.show()
  return None  


In [ ]:
!pip install gdown -q
#bundle-up-down-relooped.pdb
!gdown 1FJM4xci6nNqE93BXLnhGrdv9Dg1E03y5

# Real world XML file

The XML commenting out trick: Lines without the starting `<` are considered text and are ignored by the script parser.

```XML
<Add mover="softpack_core"/>
Add mover="softpack_bdry"/>
Add mover="softpack_surf"/>
<Add mover="hardmin_sconly"/>
```

In the above example only `softpack_core` and `hardmin_sconly` movers would be excecuted.

In [ ]:
#@title LayerDesign.xml
%%writefile LayerDesign.xml 
<ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="sfxn_soft_design" weights="beta_nov16_soft" /> 
        <ScoreFunction name="sfxn_design" weights="beta_nov16" >
            <Reweight scoretype="sap_constraint" weight="1.0" />
            <Reweight scoretype="coordinate_constraint" weight="1.0"/>
        </ScoreFunction>
        <ScoreFunction name="sfxn_design_cart" weights="beta_nov16_cart" >
            <Reweight scoretype="sap_constraint" weight="1.0" />
            <Reweight scoretype="coordinate_constraint" weight="1.0"/>
        </ScoreFunction>       
        <ScoreFunction name="sfxn_clean" weights="beta_nov16" />
        <ScoreFunction name="sfxn_clean_cart" weights="beta_nov16_cart" />
    </SCOREFXNS>

    <RESIDUE_SELECTORS>

		<SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
		<SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
		<SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
		<And name="helix_cap" selectors="entire_loop">
			<PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
		</And>
		<And name="helix_start" selectors="entire_helix">
			<PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
		</And>
		<And name="helix" selectors="entire_helix">
			<Not selector="helix_start"/>
		</And>
		<And name="loop" selectors="entire_loop">
			<Not selector="helix_cap"/>
		</And>

        <Layer name="core" use_sidechain_neighbors="true" core_cutoff="4.9" surface_cutoff="2.7" sc_neighbor_dist_exponent="1"  select_core="true" />
		<Layer name="bdry" use_sidechain_neighbors="true" core_cutoff="4.9" surface_cutoff="2.7" sc_neighbor_dist_exponent="1"  select_boundary="true" />		
		<Layer name="surf" use_sidechain_neighbors="true" core_cutoff="4.9" surface_cutoff="2.7" sc_neighbor_dist_exponent="1"  select_surface="true" />

    </RESIDUE_SELECTORS>

    <RESIDUE_LEVEL_TASK_OPERATIONS>
        <PreventRepackingRLT name="fix" />
        <RestrictToRepackingRLT name="repack" />
        <IncludeCurrentRLT name="design" />
    </RESIDUE_LEVEL_TASK_OPERATIONS>    

    <TASKOPERATIONS>
       <IncludeCurrent name="ic" /> //includes input pdbs rotamers
	   <LimitAromaChi2 name="limitaro" chi2max="110" chi2min="70" include_trp="1" /> //disallow extreme aromatic rotamers
	   <ExtraRotamersGeneric name="ex1_ex2" ex1="1" ex2="1" /> //use ex1 ex2 rotamers
       <RestrictToRepacking name="repack_only" />	//for minimize/repack

       <ConsensusLoopDesign name="disallow_non_abego_aas" include_adjacent_residues="1"/>

    <DesignRestrictions name="layer_design_nc" >
			<Action selector_logic="surf AND ( helix OR sheet )"	aas="EKRQT" />
			<Action selector_logic="surf AND helix_start"			aas="DEKRNQSTP"/>
			<Action selector_logic="surf AND loop"					aas="DEKRNQSTGP" />
			<Action selector_logic="bdry AND helix"			    	aas="ILVAEKRQTWY" />
			<Action selector_logic="bdry AND helix_start"			aas="ILVADEKRNQSTWYP"/>
			<Action selector_logic="bdry AND sheet"			    	aas="ILVADEKRNQSTYW" />
			<Action selector_logic="bdry AND loop"					aas="ILVAFDEKRNQSTGPYW" />
			<Action selector_logic="core AND helix"			    	aas="ILVAF" />
			<Action selector_logic="core AND helix_start"			aas="ILVAFP"/>
			<Action selector_logic="core AND sheet"				    aas="ILVAF" />
			<Action selector_logic="core AND loop"					aas="ILVAFGP" />
			<Action selector_logic="helix_cap"						aas="DNST"/>
		</DesignRestrictions>

    <DesignRestrictions name="design_core" >
			<Action residue_level_operations="design" selector_logic="core" />
			<Action residue_level_operations="fix"    selector_logic="bdry"	 />
			<Action residue_level_operations="fix"    selector_logic="surf"	 />
		</DesignRestrictions>

    <DesignRestrictions name="design_bdry" >
			<Action residue_level_operations="repack" selector_logic="core" />
			<Action residue_level_operations="design" selector_logic="bdry"	 />
			<Action residue_level_operations="fix"    selector_logic="surf"	 />
		</DesignRestrictions>

    <DesignRestrictions name="design_surf" >
			<Action residue_level_operations="fix"    selector_logic="core" />
			<Action residue_level_operations="repack" selector_logic="bdry"	 />
			<Action residue_level_operations="design" selector_logic="surf"	 />
		</DesignRestrictions>
    </TASKOPERATIONS>

    <SIMPLE_METRICS>

        <SapScoreMetric name="sap_score" />
        <PerResidueSapScoreMetric name="per_res_sap" />

        <SelectedResidueCountMetric name="num_res_all"  />
        <TotalEnergyMetric name="total_score" scoretype="total_score" scorefxn="sfxn_clean" />    
        <CalculatorMetric name="score_per_res" equation="total_score / num_res_all" >
            <Var name="total_score" metric="total_score" />
            <Var name="num_res_all" metric="num_res_all" />
        </CalculatorMetric>
        <SequenceMetric name="seq" custom_type="seq" />
    </SIMPLE_METRICS>

    <FILTERS>

        <NetCharge name="charge" chain="1" confidence="0" />

        <BuriedUnsatHbonds name="vbuns"  report_all_heavy_atom_unsats="true" scorefxn="sfxn_clean" ignore_surface_res="true" print_out_info_to_pdb="true" atomic_depth_selection="5.5" burial_cutoff="1000" confidence="0" />
		# For production setting dalpha_ball=1 is recommended. Se
        <BuriedUnsatHbonds name="sbuns"  report_all_heavy_atom_unsats="true" scorefxn="sfxn_clean" cutoff="4" residue_surface_cutoff="20.0" ignore_surface_res="true" print_out_info_to_pdb="true" 
        dalphaball_sasa="0" probe_radius="1.1" atomic_depth_selection="5.5" atomic_depth_deeper_than="false" confidence="0" />
        
    </FILTERS>

    <MOVERS>

        <PackRotamersMover name="softpack_core" scorefxn="sfxn_soft_design" task_operations="design_core,layer_design_nc,ic,limitaro,ex1_ex2" />
        <PackRotamersMover name="softpack_bdry" scorefxn="sfxn_soft_design" task_operations="design_bdry,layer_design_nc,ic,limitaro,ex1_ex2" />
        <PackRotamersMover name="softpack_surf" scorefxn="sfxn_soft_design" task_operations="design_surf,layer_design_nc,ic,limitaro,ex1_ex2" />
        <PackRotamersMover name="hardpack_core" scorefxn="sfxn_design" task_operations="design_core,layer_design_nc,ic,limitaro,ex1_ex2" />
        <PackRotamersMover name="hardpack_bdry" scorefxn="sfxn_design" task_operations="design_bdry,layer_design_nc,ic,limitaro,ex1_ex2" />
        <PackRotamersMover name="hardpack_surf" scorefxn="sfxn_design" task_operations="design_surf,layer_design_nc,ic,limitaro,ex1_ex2" />
        <MinMover name="hardmin_sconly" scorefxn="sfxn_clean" bb="0" chi="1"   max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".0001"/>
        <MinMover name="hardmin_bb_sc" scorefxn="sfxn_clean" bb="1" chi="1"   max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".0001"/>
       
        TaskAwareMinMover name="min_core" chi="1" bb="0" jump="0" max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".001" scorefxn="sfxn_design_cart"
        task_operations="design_core" />
        
        TaskAwareMinMover name="task_min_all" chi="1" bb="1" jump="0" max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".001" scorefxn="sfxn_design_cart"
        task_operations="restrictions" />

        AddConstraintsToCurrentConformationMover name="add_backbone_cst" use_distance_cst="0" coord_dev="2.0" CA_only="true"/>

         
         AddSapConstraintMover name="add_sap" speed="fast" sap_goal="0"  penalty_per_sap="3"   packing_correction="0" />

    </MOVERS>

    <PROTOCOLS>

        <Add mover="softpack_core"/>
        Add mover="softpack_bdry"/>
        Add mover="softpack_surf"/>
        Add mover="hardmin_sconly"/>
        <Add mover="hardpack_core"/>
        <Add mover="hardpack_bdry"/>
        <Add mover="hardpack_surf"/>        
        <Add mover="hardmin_sconly"/>
        <Add mover="hardmin_bb_sc"/>
        <Add metrics="sap_score,per_res_sap" />        
        <Add filter="vbuns" />
        <Add filter="sbuns" />

        <Add filter="charge" />


        <Add metrics="seq,score_per_res" />
 


  
    </PROTOCOLS>
    <OUTPUT scorefxn="sfxn_clean_cart" />
</ROSETTASCRIPTS>


In [ ]:
import pyrosetta; # import the pyrosetta package to access functionality
import pyrosetta.distributed.io as io
pyrosetta.init("-corrections::beta_nov16") # must be called before any other pyrosetta functions. Can accept command line flags

## Inspecting the packer task
PyRosetta makes it very easy to inspect and interact with objects defined in the XML. For example, printing the mover packer configuration is straightforward.


In [ ]:
from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects

# Parse the script
xml = XmlObjects().create_from_file("LayerDesign.xml")
# Extract the object of interest
softpack_core = xml.get_mover('softpack_core')

pose = pyrosetta.io.pose_from_pdb('bundle-up-down-relooped.pdb')
#  Print the packer task
print(softpack_core.task_factory().create_task_and_apply_taskoperations(pose))

## Run the script


In [ ]:
# Parse the script
# xml = XmlObjects().create_from_file("LayerDesign.xml")
# get the main protocol
protocol = xml.get_mover("ParsedProtocol")


pose = pyrosetta.io.pose_from_pdb('bundle-up-down-relooped.pdb')
designed_pose = pose.clone()
protocol.apply(designed_pose)
designed_pose.dump_pdb('bundle-up-down-relooped-designed.pdb')


In [ ]:
view_single(designed_pose)
designed_pose.sequence()


In [ ]:
from pyrosetta.rosetta.core.scoring import CA_rmsd
from pyrosetta.rosetta.core.scoring import calpha_superimpose_pose

calpha_superimpose_pose(designed_pose, pose)
print(CA_rmsd(pose, designed_pose))

compare_poses(pose, designed_pose)


## Inspect the pose
Poses can be converted to a pandas Dataframe

In [ ]:
import pandas as pd
df_scores = pd.DataFrame.from_records([designed_pose.scores])

In [ ]:
# print all the available scores
list(df_scores.columns)

In [ ]:
# show some of the interesting
df_scores['total_score score_per_res charge sap_score vbuns sbuns'.split()]

## TASK 0: Chose the best core cutoff parameters


In [ ]:
import viewer3d
pose = pyrosetta.pose_from_pdb('/content/bundle-up-down-relooped.pdb')
v = viewer3d.presets.coreBoundarySurface(pose, window_size=(600, 600))
v.show()

## TASK 1: Write a simple XML script that packs the core with ALA, ILE or VAL
Hint: you can use create_from_string to write the XML inline
```python
XmlObjects().create_from_string("""
<XML script here>
""")
```

In [ ]:
### BEGIN SOLUTION
from pyrosetta.io import pose_from_pdb
pose=pose_from_pdb('/content/bundle-up-down-relooped.pdb')
xml_repack_core = XmlObjects().create_from_string(f"""
<ROSETTASCRIPTS>
<SCOREFXNS>
<ScoreFunction name="sfxn_soft_design" weights="beta_nov16_soft"/>
<ScoreFunction name="sfxn_design" weights="beta_nov16">
<Reweight scoretype="sap_constraint" weight="1.0"/>
<Reweight scoretype="coordinate_constraint" weight="1.0"/>
</ScoreFunction>
<ScoreFunction name="sfxn_design_cart" weights="beta_nov16_cart">
<Reweight scoretype="sap_constraint" weight="1.0"/>
<Reweight scoretype="coordinate_constraint" weight="1.0"/>
</ScoreFunction>
<ScoreFunction name="sfxn_clean" weights="beta_nov16"/>
<ScoreFunction name="sfxn_clean_cart" weights="beta_nov16_cart"/>
</SCOREFXNS>
<RESIDUE_SELECTORS>
<SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
<SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
<SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
<And name="helix_cap" selectors="entire_loop">
<PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
</And>
<And name="helix_start" selectors="entire_helix">
<PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
</And>
<And name="helix" selectors="entire_helix">
<Not selector="helix_start"/>
</And>
<And name="loop" selectors="entire_loop">
<Not selector="helix_cap"/>
</And>
<Layer name="core" use_sidechain_neighbors="true" core_cutoff="6" surface_cutoff="2.3" sc_neighbor_dist_exponent="1" select_core="true"/>
<Layer name="bdry" use_sidechain_neighbors="true" core_cutoff="6" surface_cutoff="2.3" sc_neighbor_dist_exponent="1" select_boundary="true"/>
<Layer name="surf" use_sidechain_neighbors="true" core_cutoff="6" surface_cutoff="2.3" sc_neighbor_dist_exponent="1" select_surface="true"/>
</RESIDUE_SELECTORS>
<RESIDUE_LEVEL_TASK_OPERATIONS>
<PreventRepackingRLT name="fix"/>
<RestrictToRepackingRLT name="repack"/>
<IncludeCurrentRLT name="design"/>
</RESIDUE_LEVEL_TASK_OPERATIONS>
<TASKOPERATIONS>
<IncludeCurrent name="ic"/>
//includes input pdbs rotamers
<LimitAromaChi2 name="limitaro" chi2max="110" chi2min="70" include_trp="1"/>
//disallow extreme aromatic rotamers
<ExtraRotamersGeneric name="ex1_ex2" ex1="1" ex2="1"/>
//use ex1 ex2 rotamers
<RestrictToRepacking name="repack_only"/>
//for minimize/repack
<ConsensusLoopDesign name="disallow_non_abego_aas" include_adjacent_residues="1"/>
<DesignRestrictions name="layer_design_nc">
<Action selector_logic="surf AND ( helix OR sheet )" aas="EKRQT"/>
<Action selector_logic="surf AND helix_start" aas="DEKRNQSTP"/>
<Action selector_logic="surf AND loop" aas="DEKRNQSTGP"/>
<Action selector_logic="bdry AND helix" aas="ILVAEKRQTWY"/>
<Action selector_logic="bdry AND helix_start" aas="ILVADEKRNQSTWYP"/>
<Action selector_logic="bdry AND sheet" aas="ILVADEKRNQSTYW"/>
<Action selector_logic="bdry AND loop" aas="ILVAFDEKRNQSTGPYW"/>
<Action selector_logic="core AND helix" aas="AIV"/>
<Action selector_logic="core AND helix_start" aas="AIV"/>
<Action selector_logic="core AND sheet" aas="AIV"/>
<Action selector_logic="core AND loop" aas="AIV"/>
<Action selector_logic="helix_cap" aas="DNST"/>
</DesignRestrictions>
<DesignRestrictions name="design_core">
<Action residue_level_operations="design" selector_logic="core"/>
<Action residue_level_operations="fix" selector_logic="bdry"/>
<Action residue_level_operations="fix" selector_logic="surf"/>
</DesignRestrictions>
<DesignRestrictions name="design_bdry">
<Action residue_level_operations="repack" selector_logic="core"/>
<Action residue_level_operations="design" selector_logic="bdry"/>
<Action residue_level_operations="fix" selector_logic="surf"/>
</DesignRestrictions>
<DesignRestrictions name="design_surf">
<Action residue_level_operations="fix" selector_logic="core"/>
<Action residue_level_operations="repack" selector_logic="bdry"/>
<Action residue_level_operations="design" selector_logic="surf"/>
</DesignRestrictions>
</TASKOPERATIONS>
<SIMPLE_METRICS>
<SapScoreMetric name="sap_score"/>
<PerResidueSapScoreMetric name="per_res_sap"/>
<SelectedResidueCountMetric name="num_res_all"/>
<TotalEnergyMetric name="total_score" scoretype="total_score" scorefxn="sfxn_clean"/>
<CalculatorMetric name="score_per_res" equation="total_score / num_res_all">
<Var name="total_score" metric="total_score"/>
<Var name="num_res_all" metric="num_res_all"/>
</CalculatorMetric>
<SequenceMetric name="seq" custom_type="seq"/>
</SIMPLE_METRICS>
<FILTERS>
<NetCharge name="charge" chain="1" confidence="0"/>
<BuriedUnsatHbonds name="vbuns" report_all_heavy_atom_unsats="true" scorefxn="sfxn_clean" ignore_surface_res="true" print_out_info_to_pdb="true" atomic_depth_selection="5.5" burial_cutoff="1000" confidence="0"/>
# For production setting dalpha_ball=1 is recommended. Se
<BuriedUnsatHbonds name="sbuns" report_all_heavy_atom_unsats="true" scorefxn="sfxn_clean" cutoff="4" residue_surface_cutoff="20.0" ignore_surface_res="true" print_out_info_to_pdb="true" dalphaball_sasa="0" probe_radius="1.1" atomic_depth_selection="5.5" atomic_depth_deeper_than="false" confidence="0"/>
</FILTERS>
<MOVERS>
<PackRotamersMover name="softpack_core" scorefxn="sfxn_soft_design" task_operations="design_core,layer_design_nc,ic,limitaro,ex1_ex2"/>
<PackRotamersMover name="softpack_bdry" scorefxn="sfxn_soft_design" task_operations="design_bdry,layer_design_nc,ic,limitaro,ex1_ex2"/>
<PackRotamersMover name="softpack_surf" scorefxn="sfxn_soft_design" task_operations="design_surf,layer_design_nc,ic,limitaro,ex1_ex2"/>
<PackRotamersMover name="hardpack_core" scorefxn="sfxn_design" task_operations="design_core,layer_design_nc,ic,limitaro,ex1_ex2"/>
<PackRotamersMover name="hardpack_bdry" scorefxn="sfxn_design" task_operations="design_bdry,layer_design_nc,ic,limitaro,ex1_ex2"/>
<PackRotamersMover name="hardpack_surf" scorefxn="sfxn_design" task_operations="design_surf,layer_design_nc,ic,limitaro,ex1_ex2"/>
<MinMover name="hardmin_sconly" scorefxn="sfxn_clean" bb="0" chi="1" max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".0001"/>
<MinMover name="hardmin_bb_sc" scorefxn="sfxn_clean" bb="1" chi="1" max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".0001"/>
TaskAwareMinMover name="min_core" chi="1" bb="0" jump="0" max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".001" scorefxn="sfxn_design_cart" task_operations="design_core" /> TaskAwareMinMover name="task_min_all" chi="1" bb="1" jump="0" max_iter="2000" type="lbfgs_armijo_nonmonotone" tolerance=".001" scorefxn="sfxn_design_cart" task_operations="restrictions" /> AddConstraintsToCurrentConformationMover name="add_backbone_cst" use_distance_cst="0" coord_dev="2.0" CA_only="true"/> AddSapConstraintMover name="add_sap" speed="fast" sap_goal="0" penalty_per_sap="3" packing_correction="0" />
</MOVERS>
<PROTOCOLS>
<Add mover="softpack_core"/>
<Add mover="hardmin_sconly"/>
<Add mover="hardpack_core"/>
Add mover="hardmin_sconly"/>
Add mover="hardmin_bb_sc"/>
<Add metrics="sap_score,per_res_sap"/>
<Add filter="vbuns"/>
<Add filter="sbuns"/>
<Add filter="charge"/>
<Add metrics="seq,score_per_res"/>
</PROTOCOLS>
<OUTPUT scorefxn="sfxn_clean_cart"/>
</ROSETTASCRIPTS>
""")
#get PDB to run repacker
pose = pyrosetta.io.pose_from_pdb('bundle-up-down-relooped.pdb')
#create a string in XML format from xml_repack_core variable
#xml_protocol = XmlObjects().create_from_string('xml_repack_core')
#get protocols from xml and name them protocol_repacker
protocol_repacker=xml_repack_core.get_mover('ParsedProtocol')
##apply protocols form xml script writtten above
protocol_repacker.apply(pose)
print(pose.sequence())
pose.dump_pdb('test_pose')
view_single(pose)
### END SOLUTION

## TASK 2: Write a simple XML script that packs the surface with K and E.

Add also a `NetCharge` filter. Print the total score, socore per residue and the charge.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

## TASK 3: Run the LayerDesign XML on your chosen relooped designs
Use a for loop to load the pdbs and run the mover for each pose.

Hint:

Use the following code to loop over pdb files:
```python
from glob import glob
for a_file in glob('*.pdb'):
  print(a_file)
```

In [ ]:
from glob import glob
pdb_list = glob('*.pdb')

In [ ]:
### BEGIN SOLUTION
from glob import glob
from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects
xml = XmlObjects().create_from_file("LayerDesign.xml")
protocol = xml.get_mover("ParsedProtocol")
for pdb in pdb_list:
  print(pdb)
  
  # Extract the object of interest

  pose = pyrosetta.io.pose_from_pdb(pdb)
  #  Print the packer task
  try:
    protocol.apply(pose)
    pose.dump_pdb('designed_'+pdb)
  except:
    print("There was a problem with "+pdb)
### END SOLUTION